In [ ]:
import sys
sys.path.append('..')

import numpy as np
import lasagne
import theano
import theano.tensor as T 
import networks

import os
import tiff_helpers

%matplotlib inline 
import matplotlib.pyplot as plt
plt.style.use('seaborn-paper')

In [ ]:
# get image path
load_folder = '/media/arimorcos/SSD 1/AM144/141203'
test_tiff = 'AM144_001_001.tif'
load_tiff = os.path.join(load_folder, test_tiff)

# load in red image 
num_pages = tiff_helpers.get_num_tiff_pages(load_tiff)
red_test = tiff_helpers.read_tiff(load_tiff, range(1, num_pages, 2)).astype(theano.config.floatX)
red_test = tiff_helpers.mean_center_img(red_test)

In [ ]:
fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(111)
ax.imshow(red_test[:, :, 100], cmap='gray');

In [ ]:
# Build network 
reload(networks)
network = networks.stn(batch_size=16, save_every=100000)

In [ ]:
# Format a test image appropriately 
input_imgs = red_test[:, :, np.array([0, 500])]
test_input = np.expand_dims(np.transpose(input_imgs, axes=(2, 0, 1)), 3)
test_input = np.tile(test_input, (1, 1, 1, 16))
test_input = np.transpose(test_input, axes=(3, 0, 1, 2))

# format test reference 
test_ref = np.expand_dims(red_test[:, :, 500], 2)
test_ref = np.tile(test_ref, (1, 1, 16))
test_ref = np.transpose(test_ref, axes=(2, 0, 1))
test_ref.shape

In [ ]:
# Train network 
reset = True
num_epochs = 100000
if reset:
    cost = []
    network.set_log_dir('/media/arimorcos/4TB External/stn_conv_net/overfitting_test')
print_every = 5

for epoch in range(num_epochs):
    temp_cost = network.train_adam(test_input, test_ref)
    
    cost.append(temp_cost)
    
    if epoch % print_every == 0:
        print("Epoch: {} | Cost: {:.5f}".format(epoch, temp_cost.tolist()))

In [ ]:
input_imgs = red_test[:, :, np.array([0, 500])]
test_input_big = np.expand_dims(np.transpose(input_imgs, axes=(2, 0, 1)), 3)
test_input_big = np.tile(test_input_big, (1, 1, 1, 16))
test_input_big = np.transpose(test_input_big, axes=(3, 0, 1, 2))

In [ ]:
0.031*100000

In [ ]:
%%time
test_out = network.process(test_input_big)
# test_out.shape

In [ ]:
aligned = test_out[0, :, :]
aligned.shape

In [ ]:
%matplotlib qt
start = 100
stop = 400

fig = plt.figure(figsize=(10, 20))
ax = fig.add_subplot(211)
tiff_helpers.imshowpair(test_ref[0, start:stop, start:stop], test_input[0, 0, start:stop, start:stop], ax)
ax = fig.add_subplot(212)
tiff_helpers.imshowpair(test_ref[0, start:stop, start:stop], aligned[start:stop, start:stop], ax)